In [1]:
import stan
import pandas as pd
import nest_asyncio
import numpy as np

nest_asyncio.apply()

/var/folders/w0/mvntm8ws53v7w48x57wdnh9w0000gn/T/ipykernel_50574/3582402879.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
schools_code = """

data {
  int<lower=0> T;   // # time points (equally spaced)
  vector[T] y;      // mean corrected return at time t
  vector[T] X;
  
  vector[T] D_sun;
  vector[T] D_sat;
  vector[T] D_mon;
}

parameters {
  real mu;
  real psi;
  real d_sat;
  real d_sun;
  real d_mon;

  vector[T] epsilon_1;
  vector[T] epsilon_2;

  real<lower=0> eta_u;
  real<lower=0> eta_d;

  real kappa_hs;
  real<lower=0> omega_h;
  real<lower=-1,upper=1> phi_h;
  real theta_h;

  // vector[3] alpha;
  array[T] simplex[3] q;

  vector[T] J_s;
  vector[T] h_s;
}
transformed parameters { 
  vector[T] J = J_s;
  vector[T] h = h_s;
  vector<lower=0>[T] xi_d;
  vector<lower=0>[T] xi_u;
  real kappa_h;

  if (kappa_hs > 2) {
    kappa_h = 0;
  }
  else if (kappa_hs < 0) {
    kappa_h = 0;
  }
  else {
    kappa_h = kappa_hs;
  }

  for (t in 1:T) {
    xi_d[t] = exp(eta_d);
    xi_u[t] = exp(eta_u);
  }

  for (t in 1:T) {
    J[t] = -xi_d[t] * q[t][1] + 0 * q[t][2] + xi_u[t] * q[t][3];
  }

  h[1] = 0;
  for (t in 2:T) {
    h[t] = h[t-1] + kappa_h * (theta_h - h[t-1]) + phi_h * epsilon_1[t] + sqrt(omega_h) * epsilon_2[t];
  }
}
model {
  // Prior
  mu ~ normal(0, 10);
  psi ~ normal(0, 10);

  d_sat ~ normal(0, 10);
  d_sun ~ normal(0, 10);
  d_mon ~ normal(0, 10);

  kappa_hs ~ normal(1, 6);

  omega_h ~ inv_gamma(3, 0.05);
  phi_h ~ normal(0, 0.5 * omega_h);
  theta_h ~ normal(0, 10);

  eta_u ~ inv_gamma(1.86, 0.43);
  eta_d ~ inv_gamma(1.86, 0.43);

  //alpha = rep_vector(1, 3);
  for (t in 1:T) {
    q[t] ~ dirichlet(rep_vector(1, 3));
  }

  for (t in 1:T) {
    epsilon_1[t] ~ normal(0, 1);
    epsilon_2[t] ~ normal(0, 1);
  }
}
generated quantities {  
  vector[T] y_rep = y;
  for (t in 2:T) {
    y_rep[t] = y_rep[t-1] + mu + psi * X[t] + d_sat * D_sat[t] + d_sun * D_sun[t] + d_mon * D_mon[t] + epsilon_1[t] * sqrt(exp(h[t-1])) + J[t];
    //+ psi * X[t]
  }
}
"""

data = { 
    "T": 729,
    "y": pd.read_csv('/Users/earl/Documents/WORK/Thesis/data/hour_4.csv')['log_diff'].to_numpy()[1:],
    "X": np.squeeze(pd.read_csv('/Users/earl/Documents/WORK/Thesis/data/log_temp.csv', usecols=['log_temp']).to_numpy()),

    "D_sun": np.squeeze(pd.read_csv('/Users/earl/Documents/WORK/Thesis/data/sundays.csv').to_numpy()[:-1]),
    "D_sat": np.squeeze(pd.read_csv('/Users/earl/Documents/WORK/Thesis/data/saturdays.csv').to_numpy()[:-1]),
    "D_mon": np.squeeze(pd.read_csv('/Users/earl/Documents/WORK/Thesis/data/mondays.csv').to_numpy()[:-1])
    }

posterior = stan.build(schools_code, data=data)
fit = posterior.sample(num_chains=4, num_samples=1000)
df = fit.to_frame()  # pandas `DataFrame, requires pandas

Building...


Building: found in cache, done.Messages from stanc:
Warning in '/var/folders/w0/mvntm8ws53v7w48x57wdnh9w0000gn/T/httpstan_08b1wmip/model_ou7qfcgu.stan', line 79, column 25: Argument
    0.05 suggests there may be parameters that are not unit scale; consider
    rescaling with a multiplier (see manual section 22.12).
Warning in '/var/folders/w0/mvntm8ws53v7w48x57wdnh9w0000gn/T/httpstan_08b1wmip/model_ou7qfcgu.stan', line 47, column 7: A
    control flow statement depends on parameter(s): kappa_hs.
Warning in '/var/folders/w0/mvntm8ws53v7w48x57wdnh9w0000gn/T/httpstan_08b1wmip/model_ou7qfcgu.stan', line 44, column 2: A
    control flow statement depends on parameter(s): kappa_hs.
    calculation.
    calculation.
Sampling:   0%
Sampling:   0% (1/8000)
Sampling:   0% (2/8000)
Sampling:   0% (3/8000)
Sampling:   0% (4/8000)

AttributeError: 'NoneType' object has no attribute 'to_frame'

In [ ]:
df.head()

parameters,lp__,accept_stat__,stepsize__,treedepth__,n_leapfrog__,divergent__,energy__,mu,psi,d_sat,...,y_rep.720,y_rep.721,y_rep.722,y_rep.723,y_rep.724,y_rep.725,y_rep.726,y_rep.727,y_rep.728,y_rep.729
draws,,,,,,,,,,,,,,,,,,,,,
0,-4011.616602,0.878646,0.174968,10.0,1023.0,0.0,6180.418253,9.732776,-11.907854,6.191493,...,-21649.728427,-21679.952075,-21706.218125,-21727.093949,-21747.721174,-21777.771770,-21806.875095,-21835.251863,-21867.823219,-21894.855038
1,-4010.405120,0.710051,0.164705,10.0,1023.0,0.0,6174.058467,14.244445,-6.389647,-8.538262,...,-11246.422369,-11253.806380,-11269.572290,-11283.941379,-11319.531205,-11328.377911,-11335.871939,-11342.394206,-11351.022933,-11367.208604
2,-3980.398615,0.958452,0.204285,10.0,1023.0,0.0,6084.309780,19.778694,8.013635,9.397539,...,35443.203152,35492.006730,35548.341899,35594.262074,35634.721812,35681.999494,35727.539667,35773.579899,35822.144287,35879.315425
3,-3986.104250,0.685425,0.168474,10.0,1023.0,0.0,6137.579139,14.138982,7.028010,-10.129075,...,29767.483347,29791.912405,29784.198160,29844.244335,29903.627967,29942.687069,29969.204919,30002.547342,30077.480783,30097.763844
4,-4027.533912,0.943568,0.174968,10.0,1023.0,0.0,6239.400301,-6.492083,6.830078,-7.344332,...,12911.109405,12928.311790,12934.565971,12963.572943,12977.051546,12995.433647,13009.235186,13024.754061,13039.923198,13046.823310


In [ ]:
df['y_rep.100']

draws
0      -2480.632780
1      -1306.320565
2       4554.245941
3       3864.266077
4       1531.227477
           ...     
3995    -966.569216
3996   -1831.717863
3997   -2793.034465
3998     424.703996
3999    -678.263299
Name: y_rep.100, Length: 4000, dtype: float64

In [ ]:
pd.options.display.max_seq_items = 10000
print(df.columns)

Index(['lp__', 'accept_stat__', 'stepsize__', 'treedepth__', 'n_leapfrog__',
       'divergent__', 'energy__', 'mu', 'psi', 'd_sat', 'd_sun', 'd_mon',
       'epsilon_1.1', 'epsilon_1.2', 'epsilon_1.3', 'epsilon_1.4',
       'epsilon_1.5', 'epsilon_1.6', 'epsilon_1.7', 'epsilon_1.8',
       'epsilon_1.9', 'epsilon_1.10', 'epsilon_1.11', 'epsilon_1.12',
       'epsilon_1.13', 'epsilon_1.14', 'epsilon_1.15', 'epsilon_1.16',
       'epsilon_1.17', 'epsilon_1.18', 'epsilon_1.19', 'epsilon_1.20',
       'epsilon_1.21', 'epsilon_1.22', 'epsilon_1.23', 'epsilon_1.24',
       'epsilon_1.25', 'epsilon_1.26', 'epsilon_1.27', 'epsilon_1.28',
       'epsilon_1.29', 'epsilon_1.30', 'epsilon_1.31', 'epsilon_1.32',
       'epsilon_1.33', 'epsilon_1.34', 'epsilon_1.35', 'epsilon_1.36',
       'epsilon_1.37', 'epsilon_1.38', 'epsilon_1.39', 'epsilon_1.40',
       'epsilon_1.41', 'epsilon_1.42', 'epsilon_1.43', 'epsilon_1.44',
       'epsilon_1.45', 'epsilon_1.46', 'epsilon_1.47', 'epsilon_1.48',
      